In [98]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

이미지가 0, 1, 2, ... , 255 까지 값을 가지는 배열이므로 255 로 스케일링 해서 0 ~ 1 사이 값을 가지고 하는 것임. 이렇게 해야 활성화함수 및 오류역전파 알고리즘이 잘 동작함.

In [99]:
# 랜덤시드 고정시키기
np.random.seed(3)

# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'warehouse/handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'warehouse/handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


24 * 24 이미지가 3 * 3 사이즈 32개 컨볼루션 레이어를 통과하면 (22, 22, 32) 가 됨.

이게 다시 같은 사이즈의 64개 컨볼루션 레이어를 통과하면 (20, 20, 64) 가 됨.

이게 (2, 2) 즉 50% 로 줄이는 맥스풀링 레이어를 통과하면 (10, 10, 64) 가 됨.

플래튼 레이어를 통과하면 10 * 10 * 64 = 6400 개 배열이 됨.

In [100]:
# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

데이터 생성 시 배치사이즈를 3으로 했으므로 학습 시 한 세대 당 스텝 개수는 15로 한다. 그래야 샘플 개수 3 * 15 = 45 가 됨.

In [101]:
# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=50,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/50
15/15 [==============================] - 1s 63ms/step - loss: 1.1805 - acc: 0.6444 - val_loss: 0.6113 - val_acc: 0.9333
Epoch 2/50
15/15 [==============================] - 0s 19ms/step - loss: 0.1983 - acc: 0.9778 - val_loss: 0.1154 - val_acc: 0.9333
Epoch 3/50
15/15 [==============================] - 0s 19ms/step - loss: 0.0096 - acc: 1.0000 - val_loss: 0.0687 - val_acc: 1.0000
Epoch 4/50
15/15 [==============================] - 0s 20ms/step - loss: 9.3311e-04 - acc: 1.0000 - val_loss: 0.0362 - val_acc: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 17ms/step - loss: 3.5599e-04 - acc: 1.0000 - val_loss: 0.0221 - val_acc: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 19ms/step - loss: 2.2187e-04 - acc: 1.0000 - val_loss: 0.0229 - val_acc: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 20ms/step - loss: 1.8752e-04 - acc: 1.0000 - val_loss: 0.0213 - val_acc: 1.0000
Epoch 8/50
15/15 [==============================] - 0s 19ms/step - 

In [102]:
# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
print(scores)

-- Evaluate --
acc: 100.00%
[0.01026350405991252, 1.0]


In [118]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
        'warehouse/handwriting_shape/val',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 15 images belonging to 3 classes.


In [122]:
# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(val_generator, steps=5)
output2 = np.argmax(output, axis=-1)
#np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
#print(val_generator.class_indices)
#print(val_generator.classes)
#print(val_generator.filenames)
#print("%s: " %(model.metrics_names[1]))
print(output)
print(output2)
label_map = train_generator.class_indices
print(label_map)
label_map2 = dict((v,k) for k,v in label_map.items()) #flip k,v
print(label_map2)
output3 = [label_map2[k] for k in output2]
print(output3)



-- Predict --
[[0.000 0.000 1.000]
 [1.000 0.000 0.000]
 [0.959 0.001 0.040]
 [1.000 0.000 0.000]
 [1.000 0.000 0.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.027 0.926 0.046]
 [1.000 0.000 0.000]
 [0.000 1.000 0.000]
 [0.000 1.000 0.000]
 [0.000 1.000 0.000]
 [0.027 0.926 0.046]
 [0.000 0.001 0.999]
 [0.000 1.000 0.000]]
[2 0 0 0 0 2 2 1 0 1 1 1 1 2 1]
{'rectangle': 1, 'triangle': 2, 'circle': 0}
{0: 'circle', 1: 'rectangle', 2: 'triangle'}
['triangle', 'circle', 'circle', 'circle', 'circle', 'triangle', 'triangle', 'rectangle', 'circle', 'rectangle', 'rectangle', 'rectangle', 'rectangle', 'triangle', 'rectangle']
